# Randomized Greedy Algorithm for Motif Finding

This is randomized greedy algorithm.

In [1]:
include("utils.jl")
# utils.jl has the align, getprofile, score, and probscore utility functions:

seqs = ["AGTGGATACC", # motif inserted at 4
        "AGGATATAGT", # 2
        "AGGAGGATAT", # 5
        "GCAGCGGATA", # 5
        "CATCAGGATA"] # 6
s = [1, 1, 1, 1, 1]
l = 5;

In [2]:
function motifsearch_random(sequences, l)
    """
    Given a 2D array of strings *sequences*, randomized greedy 
    search for a motif of length *l*.
    Return an array of indices corresponding to the 
    alignment found with the highest score.
    """
    # TODO: Check that l <= sequence length
    
    n = length(sequences[1])
    
    # generate an initial random alignment
    s = rand(1:n-l+1, length(sequences))
    
    # form profile P from s
    P = getprofile(align(sequences, s, l))
    
    bestscore = 0
    newscore = score(P)
    while newscore > bestscore
        bestscore = newscore
        for i = 1:length(sequences)
            # find a P-most probable l-mer from the ith sequence
            a = map(x -> probscore(x, P), [sequences[i][j:j+l-1] for j=1:n-l+1])
            # update the starting index for sᵢ
            s[i] = indmax(a)
        end
        # recalculate the profile
        P = getprofile(align(sequences, s, l))
        newscore = score(P)
    end
    
    return s
end

motifsearch_random (generic function with 1 method)

In [3]:
motif = motifsearch_random(seqs, l)
alignment = align(seqs, motif, l)
profile = getprofile(alignment)
display(motif)
display(alignment)
display(profile)
score(profile)

5-element Array{Int64,1}:
 4
 2
 5
 6
 6

5-element Array{Any,1}:
 "GGATA"
 "GGATA"
 "GGATA"
 "GGATA"
 "GGATA"

4×5 Array{Int32,2}:
 0  0  5  0  5
 0  0  0  5  0
 0  0  0  0  0
 5  5  0  0  0

25

## Evaluating the Algorithm
Simple performance metric for this randomized greedy algorithm on synthetic data.

The index error is the average number of indices that do not match the actual indices of the inserted motifs.

The score error is the average score given to profiles found by motifsearch_random minus the maximum possible score for a correct alignment.

In [8]:
include("data.jl")

errs = []
score_errs = []
t = 20
n = 300
l = 15
max_score = l*t
for i = 1:20
    
    seqs, motif, indices = gendata(t, n, l, 0)
    predicted_indices = motifsearch_random(seqs, l)
    print(length(predicted_indices))
    err = sum(indices .!= predicted_indices)

    score_err = score(seqs, predicted_indices, l) - max_score
    push!(errs, err)
    push!(score_errs, score_err)
end
@printf("Mean absolute index error: %f\n", mean(errs))
@printf("Mean absolute score error: %f\n", mean(score_errs))

2020202020202020202020202020202020202020Mean absolute index error: 19.000000
Mean absolute score error: -100.550000
